# Planeti in lune v fantazijskem svetu Vojne zvezd (Star Wars)

In [74]:
from bs4 import BeautifulSoup
import requests
from googletrans import Translator
import csv

### 1. Pridobivanje podatkov


Najprej definiramo spremenljivke, ki jih bomo potrebovali tekom naše ekstrakcije podatkov:

In [4]:
# Povezava do spletne strani
planeti_lune_url = 'https://en.wikipedia.org/wiki/List_of_Star_Wars_planets_and_moons'

# Html izbrane strani
planeti_lune_html = 'Planeti_in_lune.html'

# Mapa z izlučenimi podatki
podatki_izluščeni = 'podatki-izluščeni'


Nato pa še funkcije:

In [62]:
def prevedi_v_slovenščino(besedilo):
    prevajalnik = Translator()
    prevod = prevajalnik.translate(besedilo, src='en', dest='sl') # Iz angleščine v slovenščino
    return prevod.text

In [63]:
def pridobi_podatke(planeti_lune_url):
    try:
        response = requests.get(planeti_lune_url)
        if response.status_code == 200: # 200 pomeni, da je vse okej
            return response.text
        else: 
            raise ValueError(f'Napačna koda: {response.status_code}')
    except Exception: # Napaka
        print(f'Nalaganje strani {planeti_lune_url} ni uspelo.')

### 2. Luščenje podatkov

In [75]:
def podatki_o_planetih_in_lunah(planeti_lune_url):

    podatki = requests.get(planeti_lune_url)
    html_content = podatki.content
    juhica = BeautifulSoup(html_content, 'html.parser') # Razčlenimo
    tabela = juhica.find('table', class_='wikitable sortable')  # Tabela v Wikipediji

    podatki_wiki = []  # V ta slovar bomo zapisali izluščene podatke

    # Vrstice
    vrstice = tabela.find_all('tr')
    for vrstica in vrstice[1:]:  # Izpustimo glavo
        okvirček = vrstica.find_all('td')
        if len(okvirček) >= 5:  
            ime = okvirček[0].get_text(strip=True) # Ime planeta oz. lune
            prvič = okvirček[1].get_text(strip=True)
            leto = okvirček[2].get_text(strip=True)
            description = okvirček[4].get_text(strip=True)

            ali_je_planet = "planet" in description.lower()  # Preveri, če se koren planet pojavi v opisu
            vrsta = "planet" if ali_je_planet else "luna"  # Če se ne pojavi, izpiše luna

            prevedeno = prevedi_v_slovenščino(description) # Prevod v slovenščino

            podatki_wiki.append({   
                'Ime': ime,
                'Vrsta': vrsta,
                'Prva upodobitev': prvič,
                'Leto prve upodobitve': leto,
                'Kratek opis': prevedeno,     
            })
        
    return podatki_wiki  

In [79]:
# Test
podatki_planeti_lune = podatki_o_planetih_in_lunah(planeti_lune_url)

for element in podatki_planeti_lune:
    print(element)

{'Ime': 'Abafar', 'Vrsta': 'planet', 'Prva upodobitev': 'Star Wars: The Clone Wars', 'Leto prve upodobitve': '2012', 'Kratek opis': 'Puščavski planet, ki se nahaja v zunanjem robu s popolnoma belo površino.Planet, znan kot Praznina, je komaj poseljen, vendar je dom ogromnim količinam ridonija, redkega in hlapljivega goriva.'}
{'Ime': 'Agamar', 'Vrsta': 'planet', 'Prva upodobitev': 'Star Wars Legends:The Farlander Papers', 'Leto prve upodobitve': '1993', 'Kratek opis': 'Pust, skalnat planet v zunanjem robu.Mesto še vedno aktivnega rezervata separatističnih droidov, ki so postali znani po tem, da so preživeli vojne klonov.'}
{'Ime': 'Ahch-To', 'Vrsta': 'planet', 'Prva upodobitev': 'The Force Awakens', 'Leto prve upodobitve': '2015', 'Kratek opis': 'Velik oceanski planet, na katerem se že več let skriva Luke Skywalker, in lokacija prvega jedijevskega templja in drevesne knjižnice svetih jedijskih besedil. Porgi, lanai in thala-sirene so med domorodnimi vrstami.'}
{'Ime': 'Ajan Kloss', 'Vr

### 3. Shranjevanje podatkov

In [80]:
def save_data_to_csv(data, csv_file):
    csv_columns = ['Ime', 'Vrsta', 'Prva upodobitev', 'Leto prve upodobitve', 'Kratek opis',]

    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        writer.writerows(data)

    print('Data written to', csv_file)

csv_file = 'planeti-lune.csv'
save_data_to_csv(podatki_planeti_lune, csv_file)


Data written to planeti-lune.csv
